In [1]:
import util
from journal_api.journal_api import JournalAPI
from IPython.display import JSON
import xml.etree.ElementTree as ET
import json
import numpy as np
import pandas as pd
import logging
import yaml
from journal_api import semanticscholar_api

2023-08-18 18:51:28,804 - INFO - NumExpr defaulting to 8 threads.
2023-08-18 18:51:33,662 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-08-18 18:51:33,867 - DEBUG - Creating converter from 7 to 5
2023-08-18 18:51:33,880 - DEBUG - Creating converter from 5 to 7
2023-08-18 18:51:33,881 - DEBUG - Creating converter from 7 to 5
2023-08-18 18:51:33,882 - DEBUG - Creating converter from 5 to 7


In [2]:
with open('logging_config.yaml', 'r') as config_file:
    log_config = yaml.safe_load(config_file)
logging.config.dictConfig(log_config)

In [3]:
title = "Design and Control of Roller Grasper V2 for In-Hand Manipulation"
title = "Quantum Dynamical Approach to Predicting the Optical Pumping Threshold for Lasing in Organic Materials!!!??"

In [4]:
journal = JournalAPI()
dt = journal.search(title, 2020, False)
JSON(dt)

2023-08-18 18:51:39,535 - DEBUG - ss.search:  Found! title=Quantum Dynamical Approach to Predicting the Optical Pumping Threshold for Lasing in Organic Materials
2023-08-18 18:51:39,537 - WARNING - ss._remake_tldr: No tldr!


<IPython.core.display.JSON object>

In [7]:
def source_target(source, target):
    ret = pd.DataFrame(columns=["source", "target"])
    for t in target:
        ret = ret.append({"source": source, "target": t}, ignore_index=True)
    return ret

In [5]:
def source_target_generator(title_list, level = 1, start_year = 2017, fix_reference = False):
    ret = pd.DataFrame(columns=["source", "target"])
    if(level > 0):
        logging.info(f"source_target_generator: level={level}, len={len(title_list)}")
        for index, title in enumerate(title_list):
            logging.info(f"source_target_generator: ({index+1}/{len(title_list)}) title={title}")
            src = journal.search(title, start_year, fix_reference)
            if src != None:
                st = source_target(src["title"],src["references"])
                ret = pd.concat([ret, st], ignore_index=True)
                ret = pd.concat([ret, source_target_generator(src["references"], level - 1, start_year)], ignore_index=True)
            
    return ret

In [8]:
x = source_target_generator([title], 3, 2018, False)

2023-08-18 18:52:58,862 - INFO - source_target_generator: level=3, len=1
2023-08-18 18:52:58,863 - INFO - source_target_generator: (1/1) title=Quantum Dynamical Approach to Predicting the Optical Pumping Threshold for Lasing in Organic Materials!!!??
2023-08-18 18:53:01,105 - DEBUG - ss.search:  Found! title=Quantum Dynamical Approach to Predicting the Optical Pumping Threshold for Lasing in Organic Materials
2023-08-18 18:53:01,107 - WARNING - ss._remake_tldr: No tldr!
2023-08-18 18:53:01,129 - INFO - source_target_generator: level=2, len=16
2023-08-18 18:53:01,130 - INFO - source_target_generator: (1/16) title=Quantum dynamical effects of vibrational strong coupling in chemical reactivity
2023-08-18 18:53:03,738 - DEBUG - ss.search:  Found! title=Quantum dynamical effects of vibrational strong coupling in chemical reactivity
2023-08-18 18:53:03,740 - WARNING - ss._remake_tldr: No tldr!
2023-08-18 18:53:03,741 - WARNING - ss._remake_references: Unknown year. title=Note that the degree

In [9]:
x

,source,target
0,Quantum Dynamical Approach to Predicting the O...,Quantum dynamical effects of vibrational stron...
1,Quantum Dynamical Approach to Predicting the O...,Impact of excitonic and photonic loss mechanis...
2,Quantum Dynamical Approach to Predicting the O...,Computational Selection of Thermally Activated...
3,Quantum Dynamical Approach to Predicting the O...,Numerical Study of Triplet Dynamics in Organic...
4,Quantum Dynamical Approach to Predicting the O...,Exciton‐Polaritons and Their Bose–Einstein Con...
...,...,...
5442,Nano-alignment in semiconducting polymer films...,On the Extraction of Charge Carrier Mobility i...
5443,Nano-alignment in semiconducting polymer films...,Electrical Instability Induced by Electron Tra...
5444,Nano-alignment in semiconducting polymer films...,Highly efficient and stable deep-blue emitting...
5445,Nano-alignment in semiconducting polymer films...,"High mobility, low voltage operating C60 based..."


In [11]:
x.to_csv("test.csv", index=False)

In [12]:
from collections import defaultdict
from pyvis.network import Network

net = Network("1200px", "1200px", notebook=True)

# Example dummy data
sources = x["source"]
targets = x["target"]
sources_short = [source[:100] + "..." for source in sources]
targets_short = [target[:100] + "..." for target in targets]

# Create a dictionary to store the count of each target
target_counts = defaultdict(int)
for t in targets_short:
    target_counts[t] += 10
    if t == "...":
        target_counts[t] = 1

# Create unique node labels for each occurrence of a node
unique_nodes = {}
for node in set(sources_short + targets_short):
    count = target_counts[node]
    size = count if count > 1 else 1  # Ensuring the smallest size is 1
    unique_label = f"{node} ({count})"
    net.add_node(unique_label, size=size)
    unique_nodes[node] = unique_label

# Add edges to the graph using the unique node labels
for s, t in zip(sources_short, targets_short):
    net.add_edge(unique_nodes[s], unique_nodes[t])

# Step 3: Visualize the graph in Jupyter Notebook
net.show("network_graph2.html")


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [ ]:
ref_pd = util.string_to_df(xy["refs"][1])

In [ ]:
xy_refs = generate_title_auths_refs(ref_pd)
xy_refs

In [ ]:
xy_refs["title"]

In [ ]:
json_data = xy_refs["title"].to_json(orient='records')

# Specify the output file path
output_file = 'data.json'

# Write JSON data to a file
with open(output_file, 'w') as file:
    file.write(json_data)

In [ ]:
crossref = util.crossref_search(title)
json.dumps(crossref, ensure_ascii=False)

In [ ]:
cref = crossref_search_by_doi("10.1177/0278364914559753")

In [ ]:
crossref = crossref_similar_search(title,10)
crossref

In [ ]:
st = "ALVINN: An Autonomous Land Vehicle in a Neural Network"
tt = google_search_books(st)
tt

In [ ]:
arxiv_search("A reduction of imitation learning and structured prediction to no-regret online learning")

In [ ]:
import requests
import json

def crossref_keywords_search(keywords, rows):
    """
    crossref_search
    """
    # Specify the API endpoint
    ENDPOINT = "https://api.crossref.org/works"

    # Set up the query parameters
    params = {
        "query": " OR ".join(keywords),
        "rows": rows  # Number of results to retrieve
    }

    # Send the API request
    response = requests.get(ENDPOINT, params=params, timeout=10)

    # Parse the JSON response
    data = response.json()

    paper_list = []
    # Extract the relevant information from the response
    if data["status"] == "ok":
        papers = data["message"]["items"]
        if len(papers) > 0:
            for paper in papers:
                paper_list.append(paper)
    return paper_list

keywords = ["machine learning", "deep learning", "artificial intelligence"]
dois = crossref_keywords_search(keywords, 10)
print(json.dumps(dois))

In [ ]:
" OR ".join(keywords)

In [ ]:
import requests

def search_similar_papers(keywords):
    base_url = 'https://api.crossref.org/works'
    
    # Create the query string with the specified keywords
    query = ' '.join(f'title:{keyword} abstract:{keyword}' for keyword in keywords)
    
    # Prepare the request URL
    url = f'{base_url}?query={query}&rows=10'  # Limiting the response to 10 papers
    
    try:
        # Send the HTTP GET request
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            
            # Extract the relevant information from the response
            papers = data['message']['items']
            
            # Process the papers
            for paper in papers:
                title = paper['title'][0]
                authors = ""
                if paper.get("author") != None:
                    lst_authors = paper.get("author")
                    for author in lst_authors:
                        auth_name = author["given"] + " " + author["family"]
                        authors = authors + auth_name + ", "
                
                doi = paper['DOI']
                
                # Print the information
                print(f'Title: {title}')
                print(f'Authors: {authors}')
                print(f'DOI: {doi}')
                print('---')
        else:
            print(f'Request failed with status code {response.status_code}')
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')

# Example usage
keywords = ['finger joint montions', 'spherical rolling fingertips', 'handcrafted control', 'sophisticated object manipulation', 'roller grasper']
search_similar_papers(keywords)

In [ ]:
t = [True, True, True, True]
t
if not any(t):
    print("True")

In [ ]:
for i in range(3):
    print(i)